In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html

In [2]:
# Read in data from different markets
df1 = pd.read_excel('https://github.com/danielpetterson/darkwebmarkets/raw/master/masterhydra.xlsx', index_col=0)
df2 = pd.read_excel('https://github.com/danielpetterson/darkwebmarkets/raw/master/mastersr2-1.xlsx', index_col=0)
df3 = pd.read_excel('https://github.com/danielpetterson/darkwebmarkets/raw/master/mastersr2-2.xlsx', index_col=0)

KeyError: "['url'] not found in axis"

In [6]:
# Drop url var (Used for troubleshooting)
df1 = df1.drop('url', axis=1)

# Combine dataframes
dfdnm = pd.concat([df1, df2, df3])

In [16]:
dfdnm.head()

,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date,Origin_region
Title,,,,,,,,,,,,
*New Vendor Offer* UK Blue Cheese (2 Grams),LDNGstar,25.0,0.04269,5.0,1.0,United Kingdom,NaN,Cannabis,NaN,Hydra,2014-08-09,EU
3.5 g Sample Blue Dream Greenhouse Top Shelf,TheSecretGarden,29.0,0.04953,5.0,0.0,United States,Worldwide,Cannabis,NaN,Hydra,2014-08-09,North America
1 g Sample Berry Diesel Haze Honeycomb,TheSecretGarden,45.0,0.07685,5.0,0.0,United States,Worldwide,Cannabis,NaN,Hydra,2014-08-09,North America
4 Oz. (112 g) Berry Diesel Haze Honeycomb,TheSecretGarden,2800.0,4.78183,5.0,0.0,United States,United States,Cannabis,NaN,Hydra,2014-08-09,North America
2 Oz. (56 g) Berry Diesel Haze Honeycomb,TheSecretGarden,1500.0,2.56169,5.0,0.0,United States,United States,Cannabis,NaN,Hydra,2014-08-09,North America


#### Standardise Values Across Markets
There are cases of alternative spelling and classification differences across the markets so it is best to create dictionaries to map these to standardised values. Using a dictionary is both faster than iterating over rows to replace values and will allow us to more easily add data from other sources in future.

In [9]:
region_dict = {
    
    'Unknown': 'Unknown',
    'China': 'Asia',
    'Hong Kong, (China)': 'Asia',
    'Undeclared': 'Unknown',
    'India': 'Asia',
    'Canada': 'North America',
    'United States': 'North America',
    'Netherlands': 'EU',
    'United Kingdom': 'EU',
    'Germany': 'EU',
    'Belgium': 'EU',
    'South Africa': 'Africa',
    'Australia': 'Oceania',
    'Spain': 'EU',
    'Czech Republic': 'EU',
    'Sweden': 'EU',
    'Finland': 'EU',
    'New Zealand': 'Oceania',
    'Norway': 'Non-EU Europe',
    'Poland': 'EU',
    'Austria': 'EU',
    'Switzerland': 'Non-EU Europe',
    'Denmark': 'EU',
    'Ireland': 'EU',
    'Italy': 'EU',
    'Bulgaria': 'EU',
    'Slovenia': 'EU',
    'Armenia': 'Non-EU Europe',
    'Slovakia': 'EU',
    'Latvia': 'EU',
    'France': 'EU',
    'Hungary': 'EU',
    'Singapore': 'Asia',
    'Germany\n]': 'EU',
    'Colombia': 'Latin America',
    'Malaysia': 'Asia',
    'Israel': 'Middle East',
    'Japan': 'Asia',
    'Vatican (Holy See)': 'EU',
    'Angola': 'Africa',
    'Greece': 'EU',
    'Paraguay': 'Latin America',
    'Albania': 'Non-EU Europe',
    'Panama': 'Latin America',
    'Luxembourg': 'EU',
    'Kosovo': 'Non-EU Europe',
    'Mexico': 'North America',
    'Monaco': 'EU',
    'Argentina': 'Latin America',
    'Bolivia': 'Latin America',
    'Ukraine': 'Non-EU Europe',
    'Croatia': 'EU',
    'Denmark': 'EU',
    'Lithuania': 'EU',
    'Romania': 'EU',
    'Reunion': 'Africa',
    'Saint Martin': 'Latin America',
    'Moldova': 'Non-EU Europe',
    'Central America': 'Latin America',
    'Tuvalu': 'Oceania',
    'Thailand': 'Asia',
    'Afghanistan': 'Asia',
    'European union': 'EU',
    'Belgium': 'EU',
    'Spain': 'EU', 
    'Hungary': 'EU',
    'Philippines': 'Asia'
     
}

country_dict = {
    'China': 'China',
    'Hong Kong, (China)': 'China',
    'Undeclared': 'Unknown',
    'India': 'India',
    'Canada': 'Canada',
    'United States': 'United States',
    'Netherlands': 'Netherlands',
    'United Kingdom': 'United Kingdom',
    'Germany': 'Germany',
    'Belgium': 'Belgium',
    'South Africa': 'South Africa',
    'Australia': 'Australia',
    'Spain': 'Spain',
    'Czech Republic': 'Czech Republic',
    'Sweden': 'Sweden',
    'Finland': 'Finland',
    'New Zealand': 'New Zealand',
    'Norway': 'Norway',
    'Poland': 'Poland',
    'Austria': 'Austria',
    'Switzerland': 'Switzerland',
    'Denmark': 'Denmark',
    'Ireland': 'Ireland',
    'Italy': 'Italy',
    'Bulgaria': 'Bulgaria',
    'Slovenia': 'Slovenia',
    'Armenia': 'Armenia',
    'Slovakia': 'Slovakia',
    'Latvia': 'Latvia',
    'France': 'France',
    'Hungary': 'Hungary',
    'Singapore': 'Singapore',
    'Germany\n]': 'Germany',
    'Colombia': 'Colombia',
    'Malaysia': 'Malaysia',
    'Israel': 'Israel',
    'Japan': 'Japan',
    'Vatican (Holy See)': 'Holy See',
    'Angola': 'Angola',
    'Greece': 'Greece',
    'Paraguay': 'Paraguay',
    'Albania': 'Albania',
    'Panama': 'Panama',
    'Luxembourg': 'Luxembourg',
    'Kosovo': 'Kosovo',
    'Mexico': 'Mexico',
    'Monaco': 'Monaco',
    'Argentina': 'Argentina',
    'Bolivia': 'Bolivia',
    'Ukraine': 'Ukraine',
    'Croatia': 'Croatia',
    'Denmark / UK (Top #4 Seller)': 'United Kingdom',
    'Lithuania': 'Lithuania',
    'Romania': 'Romania',
    'Reunion (FR)': 'Reunion',
    'Saint Martin (FR)': 'Saint Martin',
    'Moldova, Republic of': 'Moldova',
    'Central America': 'Central America',
    'Tuvalu': 'Tuvalu',
    'Thailand': 'Thailand',
    'Afghanistan': 'Afghanistan',
    'European union': 'EU',
    'Belgium': 'Belgium',
    'Spain': 'Spain', 
    'Hungary': 'Hungary',
    'Philippines': 'Philippines'
}

destination_dict = {
    'Worldwide': 'Worldwide',
    'Canada': 'Canada',
    'United States': 'United States',
    'Worldwide except Australia': 'Worldwide ex AUS',
    'European Union': 'EU',
    'Undeclared': 'Undeclared',
    'Australia': 'Australia',
    'Sweden': 'Sweden',
    'Germany': 'Germany',
    'United Kingdom': 'United Kingdom',
    'United States &amp; Canada': 'United States and Canada', 
    'New Zealand': 'New Zealand',
    'Norway': 'Norway',
    'Finland': 'Finland',
    'Ireland': 'Ireland',
    'China': 'China',
    'Switzerland': 'Switzerland',
    'Denmark': 'Denmark',
    'Azerbaijan': 'Azerbaijan',
    'Netherlands': 'Netherlands',
    'Italy': 'Italy',
    'France': 'France',
    'European Union / UK / Worldwide': 'Worldwide',
    'Worldwide\n ]': 'Worldwide',
    'United States & Canada': 'United States and Canada',
    'Argentina': 'Argentina',
    'Mexico': 'Mexico',
    'Japan': 'Japan',
    'Tuvalu': 'Tuvalu',
    'Belgium': 'Belgium',
    'Spain': 'Spain',
    'Hungary': 'Hungary',
    'Philippines': 'Philippines'
}

category_dict = {
    'Apparel': 'Apparel',
    'Art': 'Art',
    'Books': 'Books',
    'Computer': 'Computer Equipment',
    'Custom': 'Custom Orders',
    'Digital': 'Digital Goods',
    'Drug': 'Drugs',
    'Cannabis': 'Cannabis',
    'Dissociatives': 'Dissociatives',
    'Ecstasy': 'Ecstacy',
    'Drugs': 'Drugs',
    'Opioids': 'Opioids',
    'Other': 'Other',
    'Precursors': 'Precursors',
    'Prescription': 'Prescription',
    'Psychedelics': 'Psychedelics',
    'Steriodpeds': 'Steroids',
    'Stimulants': 'Stimulants',
    'Electronics': 'Electronics',
    'Erotica': 'Erotica',
    'Forgeries': 'Forgeries',
    'Hardware': 'Hardware',
    'Herbs': 'Supplements',
    'Jewelry': 'Jewelry',
    'Lab': 'Lab Supplies',
    'Lotteries': 'Lottery Games',
    'Medical': 'Drug Paraphernalia',
    'Money': 'Money',
    'Packaging': 'Packaging',
    'Services': 'Services',
    'Writing': 'Writing',
    'Collectibles': 'Collectibles',
    'Biotic': 'Supplements',
    'Benzos': 'Benzos',
    'Steroids': 'Steroids',
    'Tabacco': 'Tobacco',
    'Weapons': 'Weapons',
    'Others': 'Others',
    'Custom Orders': 'Custom Orders',
    'Drug Paraphernalia': 'Drug Paraphernalia',
    'Digital Goods': 'Digital Goods'
}

subcategory_dict = {
    'Materials': 'Materials',
    'Equipment': 'Equipment',
    'Orders': 'Custom Orders', 
    'Goods': 'Digital Goods',
    'Paraphernalia': 'Drug Paraphernalia',
    'Clones': 'Cuttings',
    'Concentrates': 'Concentrates',
    'Cuttings': 'Cuttings',
    'Edibles': 'Edibles',
    'Hash': 'Hash',
    'Pre': 'Weed',
    'Seeds': 'Seeds',
    'Shake': 'Shake',
    'Synthetic': 'Synthetic',
    'Topicals': 'Topical',
    'Trim': 'Trim',
    'Weed': 'Weed',
    'Dioscorea': 'Dioscorea',
    'Ketamine': 'Ketamine',
    'Mxe': 'MXE',
    'Pcp': 'PCP',
    '5mapb': '5-MAPB',
    'Butylone': 'Butylone',
    'Ethylone': 'Ethylone',
    'Mdai': 'MDAI',
    'Mda': 'MDA',
    'Mdma': 'MDMA',
    'Methylone': 'Methylone',
    'Mpa': 'MPA',
    'Pentedrone': 'Pentedrone',
    'Pills': 'Pills',
    'Ah': 'AH',
    'Heroin': 'Heroin',
    'Kratom': 'Kratom',
    'Opium': 'Opium',
    'Barbiturates': 'Barbiturates',
    'Intoxicants': 'Intoxicants',
    "Maoi's": 'MAOI',
    'Nootropics': 'Nootropics',
    "Ssri's": 'Antidepressant',
    'Supplements': 'Supplements',
    'Tobacco': 'Tobacco',
    'Analgesics': 'Analgesic',
    'Antidepressant': 'Antidepressant',
    'Generalhealth': 'General',
    'Relaxants': 'Relaxant',
    'Sildenafilcitrate': 'ED',
    'Stimulants': 'Stimulants',
    '2cfamily': '2C',
    '4acofamily': '4-ACO',
    '4hofamily': '4-HO',
    '5meofamily': '5-MEO',
    'Allad': 'AL-LAD',
    'Bufotenin': 'Bufotenin',
    'Dmt': 'DMT',
    'Dox': 'DOx',
    'Dpt': 'DPT',
    'Entheogens': 'Entheogen',
    'Lsa': 'LSA',
    'Lsd': 'LSD',
    'Lsz': 'LSZ',
    'Met': 'MET',
    'Nbome': 'NBOMe',
    'Salvia': 'Salvia',
    'Shrooms': 'Mushrooms',
    'Anabolicsteroids': 'Anabolic',
    'Antagonists': 'Antagonist',
    'Aromataseinhibitors': 'Aromatase Inhibitors', 
    'Clenbuterol': 'Clenbuterol',
    'Drostanolone': 'Anabolic',
    'Fluoxymesterone': 'Anabolic',
    'Hcg': 'HCG',
    'Humangrowthhormones': 'HGH',
    'Mesterelone': 'Anabolic',
    'Methandrostenolone': 'Anabolic',
    'Stanozolol': 'Anabolic',
    '2dpmp': '2-DPMP',
    '4emc': '4-EMC',
    '4mec': '4-MEC',
    '5apb': '5-APB',
    '6apb': '6-APB',
    'Apvp': 'APVP',
    'Cocaine': 'Cocaine',
    'Dimethocaine': 'DMC',
    'Ephedrine': 'Ephedrine',
    'Ethylphenidate': 'Ethylphenidate', 
    'Mdppp': 'MDPPP',
    'Mdpv': 'MDPV',
    'Mephedrone': 'Mephedrone',
    'Methamphetamine': 'Methamphetamine',
    'Speed': 'Amphetamine',
    'Digital': 'Digital',
    'Physical': 'Physical',
    'Supplies': 'Supplies',
    'Games': 'Games',
    'Methoxyketamine': 'Methoxyketamine',
    '3dmmc': '3-DMMC',
    '5it': '5-IT',
    'Amt': 'AMT',
    'Caffeine': 'Caffeine',
    'Tiletamine': 'Tiletamine',
    'Benzedrine': 'Benzedrine',
    'Cocaleaves': 'Coca Leaves',
    'Scopolamine': 'Scopolamine',
    'Fmcs': 'FMCS',
    'Ethylketamine': 'Ethylketamine',
    'Pentylone': 'Pentylone',
    'Agonists': 'Agonist',
    'Antidotes': 'Antidote',
    'Dxm': 'DXM',
    'Synthetics': 'Synthetic',
    'Others': 'Others',
    'Ecstasy': 'Ecstacy',
    'MDMA': 'MDMA', 
    'Opioids': 'Opioids',
    'Dissociatives': 'Dissociatives',
    'GHB': 'GHB',
    'LSD': 'LSD',
    'DMT': 'DMT',
    'Mescaline': 'Mescaline',
    'Mushrooms': 'Mushrooms',
    'Meth': 'Methamphetamine',
    'Prescription': 'Prescription', 
    'Benzos': 'Benzos',
    'Steroids': 'Steroids',
    'Services': 'Services',
    'Tabacco': 'Tobacco',
    'Weapons': 'Weapons',
    'Arms': 'Firearms',
    'Explosives': 'Explosives',
    'Ammo': 'Ammo',
    'Custom Orders': 'Custom Orders',
    'Drug Paraphernalia': 'Paraphernalia',
    'Scales': 'Scales',
    'Rolling Supplies': 'Rolling Supplies',
    'Pipes': 'Pipes',
    'Machinery': 'Machinery',
    'Bongs': 'Bongs',
    'Apparel': 'Apparel',
    'Sunglasses': 'Sunglasses',
    'Handbags': 'Handbags',
    'Watches': 'Watches',
    'Digital Goods': 'Digital',
    'E-Books': 'E-Books',
    'Money': 'Money',
    'Erotica': 'Erotica',
    'Accessories': 'Accessories',
    'OthersPsychedelics': 'Others'
}


# Map keys to values
dfdnm['Origin'] = dfdnm['Origin'].map(country_dict)
dfdnm['Origin_region'] = dfdnm['Origin'].map(region_dict)
dfdnm['Destination'] = dfdnm['Destination'].map(destination_dict)
dfdnm['Category'] = dfdnm['Category'].map(category_dict)
dfdnm['Subcategory'] = dfdnm['Subcategory'].map(subcategory_dict)

#### Convert Variables to Appropriate Types
We want to ensure that all data is recorded as the correct type to enable further manipulation.

In [12]:
# Convert column types
#dfdnm['Reviews'] = dfdnm['Reviews'].astype(int)
dfdnm['Rating'] = pd.to_numeric(dfdnm['Rating'],errors='coerce')
dfdnm['PriceUSD'] = pd.to_numeric(dfdnm['PriceUSD'],errors='coerce')
dfdnm['PriceBTC'] = pd.to_numeric(dfdnm['PriceBTC'],errors='coerce')
dfdnm['Date'] = pd.to_datetime(dfdnm['Date'], format='%Y-%m-%d')

In [13]:
dfdnm.dtypes

Sellerid                 object
PriceUSD                float64
PriceBTC                float64
Rating                  float64
Reviews                 float64
Origin                   object
Destination              object
Category                 object
Subcategory              object
Market                   object
Date             datetime64[ns]
Origin_region            object
dtype: object

#### Remove Duplicates
The person who originally scraped the data noted that some scrapes failed and had to be restarted. This is evident in data as we have many instances of duplicate listing names on the same day. It is also necessary because SilkRoad2 features listings twice, once in the general category and once in the specific subcategory. By keeping the last of the duplicate values we should be able to retain the subcategory data.

In [ ]:
dfclean = dfdnm.drop_duplicates(subset=['Title','Date', 'Market'], keep='last')

#### Checking for Missing Data
There are a large proportion of null values in both the Rating and Review columns. Silkroad2 only implemented product reviews at a later date so the earlier scrapes have no such information. Due to the changing structure of the HTML files over time some of the information from listings was incorrect with values ending up in different/incompatible columns which may explain the missingness in Origin, Destination and Origin_region.

In [ ]:
# Convert NA Reviews to zero
dfdnm['Reviews'].fillna(0, inplace=True)

# Generate heat map of missing values
sns.heatmap(dfclean.isnull(),
           yticklabels=False,
           cbar=False,
           cmap='viridis')

In [ ]:
dfclean
dfclean.to_excel(r'D:\Darkweb Data\dnmarchives\masterdarknet.xlsx', index = False)

### Market Overview

In [ ]:
dfmarkets = dfclean.copy()
# Split listings by Drug or Not
drug_list = ['Drugs', 'Cannabis', 'Dissociatives', 'Ecstacy', 'Opioids', 'Precursors', 'Prescription', 'Psychedelics', 'Steroids', 'Stimulants', 'Supplements', 'Benzos']
dfmarkets['Drugs'] = dfmarkets['Category'].isin(drug_list)

dfmarkets['Drugs'].value_counts()

In [ ]:
# Calculate proportion of drug listings per date
df_drugs = dfmarkets.groupby(['Market', 'Date']).apply(lambda dft: pd.Series({'Drugs': dft.Drugs.sum(), 'Other': (~dft.Drugs).sum()}))
df_drugs['proportion_drugs'] = df_drugs['Drugs'] / (df_drugs['Other'] + df_drugs['Drugs'])

# Remove hierarchical indexing
df_drugs = df_drugs.reset_index()

In [ ]:
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    shared_yaxes=False,
    vertical_spacing=0.06,
    specs=[[{"type": "scatter"}],
           [{"type": "scatter"}],
           [{"type": "table"}]])

fig.add_trace(
    go.Table(
        header=dict(
            values=["Market","Date","# of Drug Listings","# of Non-drug Listings","Proportion of Total"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df_drugs[k].tolist() for k in df_drugs.columns[0:]],
            align = "left")
    ),
    row=3, col=1
)

fig.append_trace(go.Scatter(
    x=df_drugs[df_drugs['Market'] == "Hydra"]['Date'],
    y=df_drugs[df_drugs['Market'] == "Hydra"]['proportion_drugs'],
    name='Hydra'
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=df_drugs[df_drugs['Market'] == "SilkRoad2"]['Date'],
    y=df_drugs[df_drugs['Market'] == "SilkRoad2"]['proportion_drugs'],
    name='SilkRoad2'
), row=2, col=1)

fig.update_yaxes(range=[0, 1], row=1, col=1)
fig.update_yaxes(range=[0, 1], row=2, col=1)
fig.update_layout(hovermode="x unified", title={
        'text': "Proportion of Total Listings Involving Drugs",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [ ]:
# Sum listings for each Category, Date, Market 
dfmarkets['Count'] = 1
df_drugs2 = dfmarkets.groupby(['Market', 'Date','Category'])['Count'].sum()

# Sum total for Date, Market
date_totals = dfmarkets.groupby(['Market', 'Date']).size().reset_index(name='counts')

# Merge and calculate proportions
cat_counts = pd.merge(df_drugs2.reset_index(), date_totals,  how='left', left_on=['Market','Date'], right_on = ['Market','Date'])
cat_counts['Prop']= cat_counts['Count']/cat_counts['counts']

# Split by market for plotting
prophy = cat_counts[cat_counts['Market'] == "Hydra"]
propsr2 = cat_counts[cat_counts['Market'] == "SilkRoad2"]

In [ ]:
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    shared_yaxes=False,
    vertical_spacing=0.06,
    specs=[[{"type": "scatter"}],
           [{"type": "scatter"}]])

fig.append_trace(go.Scatter(
    x=prophy[prophy['Category'] == "Stimulants"]['Date'],
    y=prophy[prophy['Category'] == "Stimulants"]['Prop'],
    name='Hydra - Stimulants'
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=propsr2[propsr2['Category'] == "Stimulants"]['Date'],
    y=propsr2[propsr2['Category'] == "Stimulants"]['Prop'],
    name='SilkRoad2 - Stimulants'
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=prophy[prophy['Category'] == "Psychedelics"]['Date'],
    y=prophy[prophy['Category'] == "Psychedelics"]['Prop'],
    name='Hydra - Psycehdelics'
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=propsr2[propsr2['Category'] == "Psychedelics"]['Date'],
    y=propsr2[propsr2['Category'] == "Psychedelics"]['Prop'],
    name='SilkRoad2 - Psychedelics'
), row=2, col=1)

fig.update_yaxes(range=[0, 0.5], row=1, col=1)
fig.update_yaxes(range=[0, 0.5], row=2, col=1)
fig.update_layout(hovermode="x unified", title={
        'text': "Proportion of Listings By Category",
        'y':0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [ ]:
# # Generate counts for unique vendors per date
# uniquevendor = (dfmarkets.groupby('Sellerid')
#                 .filter(lambda x: len(x) > 1)
#                 .groupby(['Market', 'Date'])
#                 .size()
#                 .reset_index(name='counts'))

In [ ]:
# app = dash.Dash()

# categories = ['Psychedelics', 'Stimulants', 'Cannabis', 'Opioids', 'Unique Vendors']
# fig_dropdown = html.Div([
#     dcc.Dropdown(
#         id='fig_dropdown',
#         options=[{'label': x, 'value': x} for x in categories],
#         value=None
#     )])
# fig_plot = html.Div(id='fig_plot')
# app.layout = html.Div([fig_dropdown, fig_plot])

# @app.callback(
# dash.dependencies.Output('fig_plot', 'children'),
# [dash.dependencies.Input('fig_dropdown', 'value')])
# def update_output(category):
#     return name_to_figure(category)

# def name_to_figure(category):
#     figure = go.Figure()
#     layout1 = figure.update_layout(hovermode="x unified", title={
#         'text': "Proportion of Total Listings By Market",
#         'y':0.9,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'})
#     if category == 'Psychedelics':
#         figure.add_trace(go.Scatter(y=prophy[prophy['Category'] == "Psychedelics"]['Prop'],
#                                    x=prophy[prophy['Category'] == "Psychedelics"]['Date'],
#                                    name='Hydra - Psychedelics'))
#         figure.add_trace(go.Scatter(y=propsr2[propsr2['Category'] == "Psychedelics"]['Prop'],
#                                    x=propsr2[propsr2['Category'] == "Psychedelics"]['Date'],
#                                    name='SilkRoad2 - Psychedelics'))
#         figure.update_yaxes(range=[0, 0.5])
#         layout1
#     elif category == 'Stimulants': 
#         figure.add_trace(go.Scatter(y=prophy[prophy['Category'] == "Stimulants"]['Prop'],
#                                    x=prophy[prophy['Category'] == "Stimulants"]['Date'],
#                                    name='Hydra - Stimulants'))
#         figure.add_trace(go.Scatter(y=propsr2[propsr2['Category'] == "Stimulants"]['Prop'],
#                                    x=propsr2[propsr2['Category'] == "Stimulants"]['Date'],
#                                    name='SilkRoad2 - Stimulants'))
#         figure.update_yaxes(range=[0, 0.5])
#         layout1
#     elif category == 'Cannabis':
#         figure.add_trace(go.Scatter(y=prophy[prophy['Category'] == "Cannabis"]['Prop'],
#                                    x=prophy[prophy['Category'] == "Cannabis"]['Date'],
#                                    name='Hydra - Cannabis'))
#         figure.add_trace(go.Scatter(y=propsr2[propsr2['Category'] == "Cannabis"]['Prop'],
#                                    x=propsr2[propsr2['Category'] == "Cannabis"]['Date'],
#                                    name='SilkRoad2 - Cannabis'))
#         figure.update_yaxes(range=[0, 0.5])
#         layout1
#     elif category == 'Opioids':
#         figure.add_trace(go.Scatter(y=prophy[prophy['Category'] == "Opioids"]['Prop'],
#                                    x=prophy[prophy['Category'] == "Opioids"]['Date'],
#                                    name='Hydra - Opioids'))
#         figure.add_trace(go.Scatter(y=propsr2[propsr2['Category'] == "Opioids"]['Prop'],
#                                    x=propsr2[propsr2['Category'] == "Opioids"]['Date'],
#                                    name='SilkRoad2 - Opioids'))
#         figure.update_yaxes(range=[0, 0.5])
#         layout1
#     elif category == 'Prescription': 
#         figure.add_trace(go.Scatter(y=prophy[prophy['Category'] == "Prescription"]['Prop'],
#                                    x=prophy[prophy['Category'] == "Prescription"]['Date'],
#                                    name='Hydra - Prescription'))
#         figure.add_trace(go.Scatter(y=propsr2[propsr2['Category'] == "Prescription"]['Prop'],
#                                    x=propsr2[propsr2['Category'] == "Prescription"]['Date'],
#                                    name='SilkRoad2 - Prescription'))
#         figure.update_yaxes(range=[0, 0.5])
#         layout1
#     elif category == 'Unique Vendors': 
#         figure.add_trace(go.Scatter(y=uniquevendor[uniquevendor['Market'] == "Hydra"]['counts'],
#                                     x=uniquevendor['Date']))
# # Need to fix. Subset?
#         figure.add_trace(go.Scatter(y=uniquevendor[uniquevendor['Market'] == 'SilkRoad2']['counts'],
#                                     x=uniquevendor['Date']))
#     return dcc.Graph(figure=figure)

# app.run_server(debug=True, use_reloader=False)

#### Cocaine Data Refinement

In [ ]:
cocaina = dfclean['Subcategory'] == 'Cocaine'
dfcoca = dfclean[cocaina]

In [ ]:
dfcoca['Origin'].value_counts()
pure = re.compile('\d+%')
dfpure = dfcoca[dfcoca['Title'].str.contains(pure)]
#dfcoca['Purity'] = 
#dfcoca['Title'].str.extract(r'\d+%')


In [ ]:
#m = dfcoca['Title'].str.contains('\d+%')
#dfcoca['Title'].str.extract('(\d+)', expand=False)


In [ ]:
#LSD
lsd = dfclean['Subcategory'] == 'LSD'
dflsd = dfclean[lsd]
# Extract unit dosage from each product listing
dose = re.compile('\d+.g')
dflsd = dflsd[dflsd['Title'].str.contains(dose)]
dflsd['Dosage'] = dflsd['Title'].str.extract(r'(\d+.g)')
dflsd['Dosage'] = dflsd['Dosage'].str.extract(r'(\d+)').astype(int)


# Clean unit values
dflsd['Title'] = [i.replace(',','') for i in dflsd['Title']]
dflsd['Title'] = [i.replace('.','') for i in dflsd['Title']]

# Extract quantity of units
quant = re.compile('\d+.*x')
dflsd = dflsd[dflsd['Title'].str.contains(quant)]
dflsd['Quantity'] = dflsd['Title'].str.extract(r'(\d+.*x)')
dflsd['Quantity'] = dflsd['Quantity'].str.extract(r'(\d+)').astype(float)

# Calculate price per unit
dflsd['Unit Price'] = dflsd['PriceUSD']/dflsd['Quantity']

# Calculate price per microgram
dflsd['ug Price'] = dflsd['Unit Price']/dflsd['Dosage'].astype(float)

# Retain only most recent listing scrape values
dflsd = dflsd.drop_duplicates(subset=['Title', 'Market'], keep='last')

In [ ]:
sns.heatmap(dflsd.isnull(),
           yticklabels=False,
           cbar=False,
           cmap='viridis')

In [ ]:
# Check for outliers in unit price
dflsd.sort_values(by='ug Price', ascending=False)

In [ ]:
# Remove outliers likely caused by improper data entry/extraction
dflsd1=dflsd[['Title','PriceUSD','Origin_region','Dosage','Unit Price','ug Price']].dropna()

# Boxplot 
sns.boxplot( x=dflsd1["Origin_region"], y=dflsd1["Unit Price"])
plt.xticks(rotation=45)
plt.show()

dflsd1.sort_values(by='ug Price', ascending=False)
dflsd1.info()